In [9]:
import numpy as np

class SimplexSolver(object):

    def __init__(self, c, A, b):
        '''
        Store the input arrays for easy manipulation, i.e. self.c
        Check input dimensions, raise ValueError if they don't line up
        Check feasibility.
        '''
        self.c = np.array(c)
        self.A = np.array(A)
        self.b = np.array(b)
        self.m = A.shape[0]
        self.n = A.shape[1]
        infeasible = False
        if b.any() < 0:
            infeasible = True
        if infeasible:
            raise NameError('Problem is infeasible')
        self.L = None
        self.tab = None

    def generateTableau(self):
        '''Generate the simplex tableau for solving the system
        
        [0,0] is the value of the objective function
        Along the top row of the tableau is the objective function
        in the form 0=-c
        
        Down the first column are the system's current constraints.
        The non-negative constraints for all variables is assumed
        
        The remaining entries in the matrix are the constraint equations
        
        Set self.tab for the tableau data structure
        Set self.vars for the variable list
        Set self.nbasic for the number of basic variables
        '''
        self.L = np.append(np.arange(self.n, self.n + self.m), np.arange(self.n))
        self.nbasic = self.m
        cbar = np.append(self.c, np.zeros((self.m,1)), axis=0)
        top = np.hstack((np.array([0]).reshape(1,1), -cbar.T, np.array([1]).reshape(1,1)))
        Abar = np.append(self.A, np.eye(self.m), axis=1)
        bottom = np.hstack((self.b, Abar, np.zeros((self.m,1))))
        self.tab = np.vstack((top, bottom))
        
    
    def _pivot_col(self):
        '''
        Return the index of the next pivot column.

        This will be the first negative coefficient of the objective function.
        '''
        i = 1
        while i <= self.tab.shape[1]:
            if self.tab[0,i] < 0:
                return i
            i += 1
        raise NameError('No appropriate pivot column found')
        

    def _pivot_row(self, col):
        '''
        Determine the index of the next pivot row using the ratio test
        '''
        options = []
        tops = []
        for i in xrange(self.tab.shape[0]-1):
            if self.tab[i+1,col] > 0:
                options.append(self.tab[i+1,col])
                tops.append(self.tab[i+1,0])
            else:
                options.append(np.pi)
                tops.append(np.inf)
        options = np.array(options)
        if options[options != np.pi] == None:
            raise NameError('The problem is unbounded and has no solutions')
        tops = np.array(tops)
        return np.argmin(tops/options)+1
        
    def _reduceform(self, tab, row, col):
        '''
        Reduce the given column to an elementary vector
        '''
        tab[row, :] /= tab[row, col]
        for i in xrange(self.tab.shape[0]):
            if i != row:
                tab[i,:] -= tab[i,col]*tab[row,:]
        self.tab = tab
        

    def _pivot(self, row=None, col=None):
        '''
        Perform a pivot in the tableau
        
        Make sure to check for infeasibility and to keep track of basic and non-basic variables
        '''
        col = self._pivot_col()
        row = self._pivot_row(col)
        enter = np.where(self.L == col - 1)
        leave = np.where(self.L == self.L[row-1])
        self.L[enter], self.L[leave] = self.L[leave], self.L[enter]
        self._reduceform(self.tab, row, col)
        print self.tab, '\n'
        
    def solve(self):
        '''
        Solve linear program by iterating pivots.

        Return a tuple in the format (max value, basic variables dictionary, nbasic variables dictionary)
        ''' 
        k = 0
        while (self.tab[0,:] < 0).any() and k < 100:
            k += 1
            self._pivot()
        biggest = self.tab[0,0]
        basics = {self.L[i]:self.tab[i+1,0] for i in xrange(self.m)}
        nonbasics = {self.L[i]:0 for i in xrange(self.m,self.m+self.n)}
        return (biggest, basics, nonbasics)

def problem6():
    A = np.array([[1,-1],[3,1],[4,3]])
    b = np.array([[2],[5],[7]])
    c = np.array([3,2]).reshape(2,1)
    thing = SimplexSolver(c, A, b)
    thing.generateTableau()
    print thing.tab
    return thing.solve()
    
def problem7():
    stuff = np.load('productMix.npz')
    c = stuff['p'].reshape(stuff['p'].shape[0], 1)
    print c.shape
    A = np.append(stuff['A'], np.eye(stuff['A'].shape[1]), axis=0)
    b1 = stuff['m'].reshape(stuff['m'].shape[0], 1)
    b2 = stuff['d'].reshape(stuff['d'].shape[0], 1)
    b = np.append(b1, b2, axis=0)
    
    thing = SimplexSolver(c, A, b)
    thing.generateTableau()
    print thing.tab.shape
    return thing.solve()

In [10]:
c = np.array([10.0,-57.0,-9.0,-24.0]).reshape(4,1)
b = np.array([[0.0],[0.0]]).reshape(2,1)
A = np.array([[0.5,-5.5,-2.5,9.0],[0.5,-1.5,-0.5,1.0]])

thing = SimplexSolver(c, A, b)
thing.generateTableau()
print thing.tab
print thing.solve()

[[  0.  -10.   57.    9.   24.   -0.   -0.    1. ]
 [  0.    0.5  -5.5  -2.5   9.    1.    0.    0. ]
 [  0.    0.5  -1.5  -0.5   1.    0.    1.    0. ]]
[[   0.    0.  -53.  -41.  204.   20.    0.    1.]
 [   0.    1.  -11.   -5.   18.    2.    0.    0.]
 [   0.    0.    4.    2.   -8.   -1.    1.    0.]] 

[[  0.     0.     0.   -14.5   98.     6.75  13.25   1.  ]
 [  0.     1.     0.     0.5   -4.    -0.75   2.75   0.  ]
 [  0.     0.     1.     0.5   -2.    -0.25   0.25   0.  ]] 

[[  0.   29.    0.    0.  -18.  -15.   93.    1. ]
 [  0.    2.    0.    1.   -8.   -1.5   5.5   0. ]
 [  0.   -1.    1.    0.    2.    0.5  -2.5   0. ]] 

[[  0.    20.     9.     0.     0.   -10.5   70.5    1.  ]
 [  0.    -2.     4.     1.     0.     0.5   -4.5    0.  ]
 [  0.    -0.5    0.5    0.     1.     0.25  -1.25   0.  ]] 

[[  0.  -22.   93.   21.    0.    0.  -24.    1. ]
 [  0.   -4.    8.    2.    0.    1.   -9.    0. ]
 [  0.    0.5  -1.5  -0.5   1.    0.    1.    0. ]] 

[[  0.   0.  27.  